<a href="https://colab.research.google.com/github/Mariannly/Quark5-CoAfina2025/blob/main/An%C3%A1lisis_hist%C3%B3rico_Riohacha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌎 Análisis Histórico de Sequías en Riohacha, Colombia

**Autores:** Mariannly Marquez, Alexa Serrano, Cristian Orduz, Jhon Almanzar y Andre Avila (Equipo Quark5).  
**Afiliación:** Universidad Industrial de Santander (UIS) y Universidad Yachay Tech   
**Objetivo:** Analizar la evolución histórica de las condiciones de sequía en Riohacha, Colombia.


In [15]:
# Instalar modulos necesarios
!pip install -q "cdsapi>=0.7.7" #API ERA5
!pip install -q xarray
!pip install -q cfgrib
!pip install -q --upgrade "xclim>=0.46" #Modulo de estadisticas climáticas
!pip install -q pymannkendall # Analisis estadistico

In [16]:
# Importar modulos necesarios
import os
import cdsapi
import time
import xarray as xr
import cfgrib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import xclim as xc
import pymannkendall as mk
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

## 📦 1. Obtención de los Datos

**Fuente:**  
- ERA5-Land monthly averaged data (Copernicus Climate Data Store)  
  Variables empleadas:
  - Precipitación total (tp)
  - Temperatura del aire a 2 m (t2m)
  - Evaporación total (e)
  - Evaporación parcial (pev)
  - Humedad del suelo (swvl1, swvl2, swvl3, swvl4)

**Periodo temporal:** 1985 – 2025    
**Área de estudio:** Riohacha, Colombia.

In [17]:
# Acceder a ERA5 con la API
cdsapirc_content = """url: https://cds.climate.copernicus.eu/api
key: 01876cbb-e8a2-4cb4-8d98-5f4ba769fc1a
"""

# Crear el archivo de configuración en la ruta esperada
with open(os.path.expanduser("~/.cdsapirc"), "w") as f:
    f.write(cdsapirc_content)

# Acceder al cliente
c = cdsapi.Client()

# Definir detalles de descarga
dataset = "reanalysis-era5-land-monthly-means"
bbox_riohacha = {
    "north": 11.80,
    "south": 11.30,
    "west": -73.20,
    "east": -72.60,
}
request = {
    "product_type": "monthly_averaged_reanalysis",
    "variable": [
        "total_precipitation",
        "2m_temperature",
        "volumetric_soil_water_layer_1",
        "volumetric_soil_water_layer_2",
        "volumetric_soil_water_layer_3",
        "volumetric_soil_water_layer_4",
        "surface_solar_radiation_downwards",
        "potential_evaporation",
        "total_evaporation"
    ],
    "year": [str(y) for y in range(1985, 2025)],   # 1985–2025
    "month": [f"{m:02d}" for m in range(1, 13)],
    "time": "00:00",
    "area": [
        bbox_riohacha["north"],
        bbox_riohacha["west"],
        bbox_riohacha["south"],
        bbox_riohacha["east"],
    ],
    "format": "netcdf",
}

# Descargar
data_file = c.retrieve(dataset, request).download()

2025-11-09 00:45:09,934 INFO Request ID is adb675f5-b694-48c8-b7cf-8a2185002808
INFO:ecmwf.datastores.legacy_client:Request ID is adb675f5-b694-48c8-b7cf-8a2185002808
2025-11-09 00:45:10,098 INFO status has been updated to accepted
INFO:ecmwf.datastores.legacy_client:status has been updated to accepted
2025-11-09 00:45:18,883 INFO status has been updated to running
INFO:ecmwf.datastores.legacy_client:status has been updated to running
2025-11-09 00:45:24,109 INFO status has been updated to successful
INFO:ecmwf.datastores.legacy_client:status has been updated to successful


befff589feaa9f29fbadef8d7540c4cb.zip:   0%|          | 0.00/417k [00:00<?, ?B/s]

In [18]:
# Descomprimir archivo
!unzip -q {data_file}
file = "data_stream-moda.nc"

# Leer y ver archivo
ds = xr.open_dataset(file)
print(ds)

<xarray.Dataset> Size: 737kB
Dimensions:     (valid_time: 480, latitude: 6, longitude: 7)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 4kB 1985-01-01 ... 2024-12-01
  * latitude    (latitude) float64 48B 11.8 11.7 11.6 11.5 11.4 11.3
  * longitude   (longitude) float64 56B -73.2 -73.1 -73.0 ... -72.8 -72.7 -72.6
    number      int64 8B ...
    expver      (valid_time) <U4 8kB ...
Data variables:
    tp          (valid_time, latitude, longitude) float32 81kB ...
    t2m         (valid_time, latitude, longitude) float32 81kB ...
    swvl1       (valid_time, latitude, longitude) float32 81kB ...
    swvl2       (valid_time, latitude, longitude) float32 81kB ...
    swvl3       (valid_time, latitude, longitude) float32 81kB ...
    swvl4       (valid_time, latitude, longitude) float32 81kB ...
    ssrd        (valid_time, latitude, longitude) float32 81kB ...
    pev         (valid_time, latitude, longitude) float32 81kB ...
    e           (valid_time, latitude, longitude) fl

## ⚙️ 2. Preprocesamiento de Datos

In [19]:
# Convertir Data, eliminar valores inncesarios
df = ds.to_dataframe()
df = df.dropna()
df.drop(["number", "expver"], axis=1, inplace=True)
df = df.reset_index()

# Asegura tipo fecha
df["valid_time"] = pd.to_datetime(df["valid_time"])

# Columnas de valor y promedio mensual
value_cols = ["t2m","swvl1","swvl2","swvl3","swvl4","ssrd","pev","e","tp"]

# Agrupar por mes (un solo punto por mes)
df = (
    df.groupby("valid_time", as_index=False)[value_cols]
      .mean()
)

In [20]:
# Convertir unidades
days = 30

# Fechas
t = pd.to_datetime(df["valid_time"]).values

# conviertir a mm/día y ajustar valores
pr_rate  = df["tp"].to_numpy() *days * 1000.0
pet_rate = -df["pev"].to_numpy() *days * 1000.0

# Empaquetar con unidades
pr  = xr.DataArray(pr_rate,  coords={"time": t}, dims=("time",), attrs={"units": "mm d-1"})
pet = xr.DataArray(pet_rate, coords={"time": t}, dims=("time",), attrs={"units": "mm d-1"})

# Restar y reasignar unidades
wb = pr - pet
wb = wb.assign_attrs(units="mm d-1")

In [21]:
# Pasar a °C
df["t2m"] = df["t2m"] - 273.15

# m³/m³ (fracción volumétrica, entre 0 y 1)
for col in ["swvl1","swvl2","swvl3","swvl4"]:
    df[col] = df[col] * 100
# mm/mes (acumulado mensual)
df["tp"] = df["tp"] * days * 1000

# mes (acumulado mensual)
df["e"] = -df["e"] * days * 1000

# mes (acumulado mensual)
df["pev"] = -df["pev"] * days * 1000

# MJ/m²/día
df["ssrd"] = df["ssrd"] / 86400.0

## 📉 3. Análisis Exploratorio e Indicadores de Sequía

In [22]:
# Calcular indicadores relevantes

# Funciones de SPI y SPEI
SPI  = getattr(xc.indices, "spi",  getattr(xc.indices, "standardized_precipitation_index"))
SPEI = getattr(xc.indices, "spei", getattr(xc.indices, "standardized_precipitation_evapotranspiration_index"))

# Calcular y guardar data
windows = [1, 3, 6, 12]
spi  = {k: SPI(pr, window=k).to_series().rename(f"SPI_{k}") for k in windows}
spei = {k: SPEI(wb=wb, window=k).to_series().rename(f"SPEI_{k}") for k in windows}

# Unir valores de SPI y SPEI
spi_df  = pd.concat(spi.values(), axis=1)
spei_df = pd.concat(spei.values(), axis=1)
indices_df = pd.concat([spi_df, spei_df], axis=1)

# Merge con la df original por fecha
df = df.set_index("valid_time").join(indices_df)
df = df.reset_index()

# quitar NaN creados por media movil
df = df.dropna()

In [23]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['e'], mode='lines', name="Evaporación Total"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['tp'], mode='lines', name="Precipitación total"))
fig.update_layout(
    title='Precipitacióin y Evaporación Total en el Tiempo',
    xaxis_title='Año',
    yaxis_title='Precipitacióin y Evaporación Total (mm/mes)',
    hovermode='x unified'
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [24]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPI_1'], mode='lines', name="SPI (k=1mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPI_3'], mode='lines', name="SPI (k=3mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPI_6'], mode='lines', name="SPI (k=6mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPI_12'], mode='lines', name="SPI (k=12mes)"))
fig.update_layout(
    title='Indice de Precipitación Estandarizado',
    xaxis_title='Año',
    yaxis_title='PSI',
    hovermode='x unified'
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [25]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPEI_1'], mode='lines', name="SPEI (k=1mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPEI_3'], mode='lines', name="SPEI (k=3mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPEI_6'], mode='lines', name="SPEI (k=6mes)"))
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPEI_12'], mode='lines', name="SPEI (k=12mes)"))
fig.update_layout(
    title='Indice de Precipitación y Evotranspiración Estandarizado',
    xaxis_title='Año',
    yaxis_title='PSI',
    hovermode='x unified'
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

### 🔍 Observaciones clave

- SPEI y SPI son dos índices de sequía.
- Índice de Precipitación Estandarizado (SPI): Es un índice de sequía que compara las **precipitaciones** observadas con los **promedios históricos** para un período de tiempo determinado. En este analisis consideramos k= 1, 3, 6 y 12 meses.
- Índice Estandarizado de Precipitación-Evapotranspiración (SPEI): Es un índice de sequía multiescalar que considera tanto la **precipitación** como la **evapotranspiración** potencial (también para k= 1, 3, 6 y 12 meses).
- En ambos casos, valores más negativos indican condiciones más severas de sequía.

## ✅ 4. Análisis de Tendencias de Sequías con Mann-Kendall

In [26]:
trend_data = []
for col in ['SPI_1', 'SPI_3', 'SPI_6', 'SPI_12', 'SPEI_1', 'SPEI_3', 'SPEI_6', 'SPEI_12']:
    series = df[col].dropna()
    if not series.empty:
        result = mk.original_test(series)
        trend_data.append({
            'Index': col,
            'Slope': result.slope,
            'P_Value': result.p,
            'Trend': result.trend,
            'Significant': result.p < 0.05
        })

trend_df = pd.DataFrame(trend_data)
print(trend_df)

     Index     Slope       P_Value       Trend  Significant
0    SPI_1 -0.001578  5.231551e-06  decreasing         True
1    SPI_3 -0.001886  1.324517e-07  decreasing         True
2    SPI_6 -0.002163  9.461569e-10  decreasing         True
3   SPI_12 -0.002510  1.081357e-13  decreasing         True
4   SPEI_1 -0.001040  3.995785e-03  decreasing         True
5   SPEI_3 -0.001135  2.552089e-03  decreasing         True
6   SPEI_6 -0.001283  5.307237e-04  decreasing         True
7  SPEI_12 -0.001566  8.408172e-05  decreasing         True


In [27]:
# Retrieve the Mann-Kendall slope for SPEI_12 from the trend_df
slope_spei12_from_df = trend_df[trend_df['Index'] == 'SPEI_12']['Slope'].iloc[0]

# Calculate the trend line
numeric_index_for_slope = np.arange(len(df['SPEI_12']))

# Determine the starting point for the trend line based on the first valid data point in the series
first_spei12_value_in_series = df['SPEI_12'].iloc[0]

# Calculate the y-values for the trend line using the slope from trend_df
trend_line_y = first_spei12_value_in_series + slope_spei12_from_df * (numeric_index_for_slope - numeric_index_for_slope[0])


fig = go.Figure()
fig.add_trace(go.Scatter(x=df["valid_time"], y=df['SPEI_12'], mode='lines', name='SPEI (k= 12 meses)'))
fig.add_trace(go.Scatter(x=df["valid_time"], y=trend_line_y, mode='lines', name='Tendencia (Mann-Kendall)', line=dict(color='red', dash='dash')))
fig.update_layout(
    title='SPEI con Línea de Tendencia Mann-Kendall',
    xaxis_title='Año',
    yaxis_title='Valor de SPEI (k=12 meses)',
    hovermode='x unified'
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()


### 🔍 Observaciones clave

- El test de Mann–Kendall evalúa si existe una tendencia monótona (creciente o decreciente) en la serie de tiempo.
- SPEI para Riohacha muestra una **pendiente negativa estadísticamente significativa (p < 0.05)**, indicando una tendencia hacia condiciones más secas en el periodo 1985–2024.
- Esto indica que las condiciones se vuelven **más secas** a lo largo del tiempo.

## 📌 5. Contextualización

Además del análisis climático cuantitativo con datos ERA5, se recopilaron reportes de prensa y boletines institucionales que reflejan los impactos sociales y ambientales de las sequías recientes en La Guajira.  
Estos eventos permiten validar el comportamiento observado en los índices de sequía y comprender mejor las afectaciones locales.

| Fecha | Evento reportado | Fuente / Observación |
|--------|------------------|----------------------|
| **5 de junio de 2025** | La temporada de lluvias (finales de abril – principios de mayo) se comportó irregularmente, con lluvias por debajo del promedio. Se atribuye al cambio climático. | *Periódicos locales (CAMBIO CLIMÁTICO)* |
| **1 de junio – 30 de noviembre (2025)** | Inicio de la temporada de ciclones tropicales que impactan directamente a La Guajira, incrementando la variabilidad climática. | *Servicio Meteorológico Nacional* |
| **Febrero de 2021** | El **23 de febrero**, el río Tapias presentó **1.300 L/s menos** de su caudal normal (3.500 L/s), reflejando una marcada disminución del recurso hídrico. | *Periódicos regionales* |
| **Fenómeno del Niño (patrón histórico)** | Normalmente se presenta entre **noviembre y abril**, con síntomas iniciales desde mitad de año en las zonas más secas de la región. | *IDEAM* |
| **Enero de 2020** | Se declara **calamidad pública por seis meses en Hatonuevo**, municipio vecino de Riohacha, debido a la escasez de agua. | *Noticias locales* |
| **Marzo – mayo de 2019** | Sequías prolongadas causan estragos en comunidades Wayúu y pérdidas de ganado. | *Medios locales* |
| **Febrero de 2019** | El río Tapia, que alimenta el acueducto de Riohacha, disminuye su nivel en más del **50%**. | *Prensa regional* |
| **2014** | Año de **sequía extrema**, con afectaciones prolongadas e irreversibles en ecosistemas y medios de vida. | *Archivo de prensa nacional* |  




In [28]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df["valid_time"][-150:], y=df['SPEI_12'][-150:], mode='lines', name='SPEI (k= 12 meses)'))
fig.add_trace(go.Scatter(x=df["valid_time"][-150:], y=trend_line_y[-150:], mode='lines', name='Tendencia (Mann-Kendall)', line=dict(color='red', dash='dash')))

fig.add_vrect(x0=pd.to_datetime('2021-02-01'), x1=pd.to_datetime('2021-02-28'), line_width=0, fillcolor="red", opacity=0.2)
fig.add_vrect(x0=pd.to_datetime('2020-01-01'), x1=pd.to_datetime('2020-06-30'), line_width=0, fillcolor="red", opacity=0.2)
fig.add_vrect(x0=pd.to_datetime('2019-03-01'), x1=pd.to_datetime('2019-04-30'), line_width=0, fillcolor="red", opacity=0.2)
fig.add_vrect(x0=pd.to_datetime('2019-02-01'), x1=pd.to_datetime('2019-02-28'), line_width=0, fillcolor="red", opacity=0.2)
fig.add_vrect(x0=pd.to_datetime('2014-01-01'), x1=pd.to_datetime('2014-12-30'), line_width=0, fillcolor="red", opacity=0.2)

# Add annotations for the vertical bars
fig.add_annotation(
    x=pd.to_datetime('2021-02-15'), y=1.0, # Adjust y-coordinate as needed to place annotation on the plot
    text="Río Tapias", showarrow=True, arrowhead=1,
    yshift=10, font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=0.5
)
fig.add_annotation(
    x=pd.to_datetime('2020-03-15'), y=0.85, # Adjust y-coordinate
    text="Calamidad pública", showarrow=True, arrowhead=1,
    yshift=10, font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=0.5
)
fig.add_annotation(
    x=pd.to_datetime('2019-04-01'), y=1.1, # Adjust y-coordinate
    text="Sequías prolongadas", showarrow=True, arrowhead=1,
    yshift=10, font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=0.5
)
fig.add_annotation(
    x=pd.to_datetime('2019-02-15'), y=0.7, # Adjust y-coordinate
    text="Río Tapia", showarrow=True, arrowhead=1,
    yshift=10, font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=0.5
)
fig.add_annotation(
    x=pd.to_datetime('2014-06-15'), y=1.0, # Adjust y-coordinate
    text="Sequía extrema 2014", showarrow=True, arrowhead=1,
    yshift=10, font=dict(size=10, color="black"), bgcolor="rgba(255,255,255,0.7)", bordercolor="black", borderwidth=0.5
)


fig.update_layout(
    title='SPEI con Línea de Tendencia Mann-Kendall ',
    xaxis_title='Año',
    yaxis_title='Valor de SPEI (k=12 meses)',
    hovermode='x unified'
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

### 🔍 Observaciones clave

- Los registros confirman una **recurrencia de eventos de sequía severa**.
- El **déficit hídrico del río Tapias** es un indicador crítico de vulnerabilidad para Riohacha y zonas Wayúu.  
- Los impactos sociales (escasez de agua, pérdida de ganado, inseguridad alimentaria) concuerdan con las **anomalías de precipitación y temperatura** observadas en el análisis ERA5.  

## 📰 Referencias



[1] Muñoz Sabater, J. (2019): ERA5-Land monthly averaged data from 1950 to present. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). DOI: 10.24381/cds.68d2bb30 (Accessed on 08-11-2025)

[2] Caballero Fernández, D., Salvador Franch, F., Padrón Padrón, P. A., & Salvà Catarineu, M. (2025). Uso de datos de reanálisis climático del ERA5 y el SPI para el estudio de las sequías (1941-1970) en El Hierro (Islas Canarias). Investigaciones Geográficas, (84), 89-112. https://doi.org/10.14198/INGEO.29359

[3] Erika Londoño, Daniel Criollo, Luis González (2023). Crisis Humanitaria en la Guajira: Las intervenciones estratégicas del Gobierno. Departamento Nacional de Planeación.

[4] Hererra posada D. Aristizábal E “Artificial Intelligence and Machine Learning Model for Spatial and Temporal
Prediction of Drought Events in the Department of Magdalena, Colombia”. DOI:
http://doi.org/10.17981/ingecuc.18.2.2022.20

[5] Eduardo León, Carlos Acosta (2015). Análisis de Vulnerabilidad del Territorio por Sequía en el Departamento de la Guajira, Colombia, a partir de una visión basada en necesidades básicas insatisfechas.  https://files.core.ac.uk/download/pdf/71895641.pdf

[6] Boletín 1 del Consejo Nacional del Agua 2023. https://www.minambiente.gov.co/boletin-no-1-del-consejo-nacional-del-agua-preparacion-para-la-alta-probabilidad-de-ocurrencia-del-fenomeno-del-nino-2023